# Example of Training with MotionSense

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
from pathlib import Path

sys.path.append("..")

## Read Processed Data

In [3]:
from librep.utils.dataset import PandasDatasetsIO

2022-08-26 18:50:08.593119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-26 18:50:08.593140: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
dataset_path = Path("../data/views/MotionSense/balanced_view")
train, validation, test = PandasDatasetsIO(dataset_path).load()

In [5]:
train = pd.concat([train, validation], ignore_index=True)
train

,Unnamed: 0,attitude.roll-0,attitude.roll-1,attitude.roll-2,attitude.roll-3,attitude.roll-4,attitude.roll-5,attitude.roll-6,attitude.roll-7,attitude.roll-8,...,userAcceleration.z-145,userAcceleration.z-146,userAcceleration.z-147,userAcceleration.z-148,userAcceleration.z-149,activity code,length,trial_code,index,user
0,0,1.311800,1.309805,1.294033,1.259262,1.214031,1.174594,1.150417,1.126066,1.071678,...,0.198949,-0.241833,-0.228292,-0.409867,-0.227758,0,150,11,300,16
1,1,0.979769,0.853751,0.724747,0.620533,0.563019,0.546236,0.540058,0.531511,0.509747,...,0.061945,0.108357,0.042498,-0.119922,-0.535207,0,150,1,750,7
2,2,2.457231,2.508876,2.562549,2.610262,2.646260,2.662423,2.663410,2.662757,2.656153,...,0.389712,-0.012963,-0.117823,-0.242463,-0.520011,0,150,1,750,11
3,3,-0.816211,-0.847936,-0.773849,-0.642674,-0.511272,-0.443049,-0.422701,-0.404203,-0.357625,...,1.096083,0.919155,0.980044,0.167161,0.291327,0,150,1,450,12
4,4,0.093224,0.153045,0.230516,0.329710,0.430513,0.511403,0.596036,0.689030,0.762821,...,0.559331,0.268818,0.286077,0.244404,0.149644,0,150,1,150,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,679,1.386363,1.278024,1.215004,1.158732,1.104921,1.074311,1.100172,1.213943,1.379470,...,-0.825850,-0.556961,0.106899,-0.041223,-0.097716,5,150,9,3000,2
3998,680,-2.299390,-1.592883,-0.645613,-0.067135,0.084786,0.149636,0.155141,0.154149,0.126544,...,-0.742960,-0.484808,-0.416761,-0.371416,0.653042,5,150,9,3750,6
3999,681,0.982494,1.130647,1.275658,1.381327,1.497785,1.592056,1.681864,1.732524,1.707926,...,-0.149763,0.592899,0.742044,-0.901616,0.222152,5,150,9,1800,2
4000,682,0.460114,0.413727,0.364852,0.367047,0.532375,1.072929,1.662109,1.712763,1.541528,...,0.406465,-0.081973,-0.163481,-0.046614,0.042567,5,150,9,2850,6


## Create the MotionSense Dataset

In [6]:
from librep.datasets.multimodal import PandasMultiModalDataset

In [7]:
features = [
    "userAcceleration.x",
    "userAcceleration.y",
    "userAcceleration.z",
    "rotationRate.x",
    "rotationRate.y",
    "rotationRate.z"
]

train_dataset = PandasMultiModalDataset(
    train,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)
validation_dataset = PandasMultiModalDataset(
    validation,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)
test_dataset = PandasMultiModalDataset(
    test,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

In [8]:
train_dataset[0]

(array([ 1.850310e-01,  1.323820e-01,  8.863600e-02,  8.935600e-02,
         1.302640e-01,  1.320740e-01,  1.471270e-01,  1.316310e-01,
        -1.874900e-02, -7.266600e-02,  2.075100e-02, -1.878720e-01,
        -3.334510e-01, -4.484450e-01, -1.459800e-01,  2.271810e-01,
         2.504250e-01,  2.776350e-01,  1.516580e-01, -2.107200e-02,
         5.909000e-02, -2.750900e-02,  4.955300e-02,  1.133940e-01,
         1.298460e-01,  3.934200e-02, -2.773300e-02, -4.913900e-02,
        -1.986110e-01, -3.969930e-01, -5.211840e-01, -3.820190e-01,
        -2.566590e-01, -6.048700e-02,  1.571870e-01,  2.215400e-01,
         2.895000e-01,  2.309840e-01, -1.455400e-01, -6.067200e-02,
         3.323650e-01,  5.399100e-02, -7.359100e-02, -2.317620e-01,
        -1.721370e-01, -7.184400e-02, -1.080600e-01,  5.770800e-02,
         2.087240e-01,  2.547080e-01,  2.270680e-01,  1.223900e-02,
        -8.920400e-02, -1.924300e-01, -2.729570e-01, -1.280950e-01,
        -1.683400e-02, -8.303300e-02,  1.845200e

In [9]:
train_dataset.window_slices, train_dataset.window_names, train_dataset.num_windows

([(0, 150), (150, 300), (300, 450), (450, 600), (600, 750), (750, 900)],
 ['userAcceleration.x',
  'userAcceleration.y',
  'userAcceleration.z',
  'rotationRate.x',
  'rotationRate.y',
  'rotationRate.z'],
 6)

In [10]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [11]:
fft_transform = FFT()
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")
train_dataset_fft = transformer(train_dataset)
validation_dataset_fft = transformer(validation_dataset)
test_dataset_fft = transformer(test_dataset)
train_dataset_fft.window_slices, train_dataset_fft.window_names, train_dataset_fft.num_windows

([(0, 150), (150, 300), (300, 450), (450, 600), (600, 750), (750, 900)],
 ['fft.userAcceleration.x',
  'fft.userAcceleration.y',
  'fft.userAcceleration.z',
  'fft.rotationRate.x',
  'fft.rotationRate.y',
  'fft.rotationRate.z'],
 6)

In [12]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT
from librep.transforms.stats import StatsTransform

fft_transform = FFT()
stats_transform = StatsTransform()
transformer = TransformMultiModalDataset(transforms=[fft_transform, stats_transform], new_window_name_prefix="fft-stats.")
train_dataset_fft_stats = transformer(train_dataset)
validation_dataset_fft_stats = transformer(validation_dataset)
test_dataset_fft_stats = transformer(test_dataset)
train_dataset_fft_stats.window_slices, train_dataset_fft_stats.window_names, train_dataset_fft_stats.num_windows

([(0, 19), (19, 38), (38, 57), (57, 76), (76, 95), (95, 114)],
 ['fft-stats.userAcceleration.x',
  'fft-stats.userAcceleration.y',
  'fft-stats.userAcceleration.z',
  'fft-stats.rotationRate.x',
  'fft-stats.rotationRate.y',
  'fft-stats.rotationRate.z'],
 6)

In [13]:
from librep.datasets.multimodal import combine_multi_modal_datasets
combined_train_dset = combine_multi_modal_datasets(train_dataset_fft, train_dataset_fft_stats)
combined_test_dset = combine_multi_modal_datasets(test_dataset_fft, test_dataset_fft_stats)
combined_train_dset.window_slices, combined_train_dset.window_names, combined_train_dset.num_windows

([(0, 150),
  (150, 300),
  (300, 450),
  (450, 600),
  (600, 750),
  (750, 900),
  (900, 919),
  (919, 938),
  (938, 957),
  (957, 976),
  (976, 995),
  (995, 1014)],
 ['fft.userAcceleration.x',
  'fft.userAcceleration.y',
  'fft.userAcceleration.z',
  'fft.rotationRate.x',
  'fft.rotationRate.y',
  'fft.rotationRate.z',
  'fft-stats.userAcceleration.x',
  'fft-stats.userAcceleration.y',
  'fft-stats.userAcceleration.z',
  'fft-stats.rotationRate.x',
  'fft-stats.rotationRate.y',
  'fft-stats.rotationRate.z'],
 12)

In [14]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT
from librep.transforms.resampler import SimpleResampler
from librep.transforms.stats import StatsTransform

resampler = SimpleResampler(new_sample_size=100)
fft_transform = FFT()
stats_transform = StatsTransform()
transformer = TransformMultiModalDataset(transforms=[resampler, fft_transform], new_window_name_prefix="resample-100-fft.")
resampled_train_dataset_fft_stats = transformer(train_dataset)
resampled_validation_dataset_fft_stats = transformer(validation_dataset)
resampled_test_dataset_fft_stats = transformer(test_dataset)
resampled_train_dataset_fft_stats.window_slices, resampled_train_dataset_fft_stats.window_names, resampled_train_dataset_fft_stats.num_windows

([(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)],
 ['resample-100-fft.userAcceleration.x',
  'resample-100-fft.userAcceleration.y',
  'resample-100-fft.userAcceleration.z',
  'resample-100-fft.rotationRate.x',
  'resample-100-fft.rotationRate.y',
  'resample-100-fft.rotationRate.z'],
 6)

## Train

In [16]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [17]:
result = multi_run_experiment(train_dataset, [test_dataset, validation_dataset, test_dataset])
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661540173.398432
    result:
    -   accuracy: 0.8234714003944773
        f1 score (macro): 0.8219762572277247
        f1 score (micro): 0.8234714003944773
        f1 score (weighted): 0.82496654356123
    -   accuracy: 1.0
        f1 score (macro): 1.0
        f1 score (micro): 1.0
        f1 score (weighted): 1.0
    -   accuracy: 0.8234714003944773
        f1 score (macro): 0.8219762572277247
        f1 score (micro): 0.8234714003944773
        f1 score (weighted): 0.82496654356123
    run id: 1
    start: 1661540166.8985915
    time taken: 6.499840497970581
-   end: 1661540179.7175586
    result:
    -   accuracy: 0.8264299802761341
        f1 score (macro): 0.8244892200246031
        f1 score (micro): 0.8264299802761341
        f1 score (weighted): 0.8283707405276651
    -   accuracy: 1.0
        f1 score (macro): 1.0
        f1 score (micro): 1.0
        f1 score (weighted): 1.0
    -   accuracy: 0.8264299802761341
        f1 score (macro): 0.8244892200246031
    

In [18]:
result = multi_run_experiment(train_dataset_fft, test_dataset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661540192.0928497
    result:
    -   accuracy: 0.8303747534516766
        f1 score (macro): 0.8328102925145252
        f1 score (micro): 0.8303747534516766
        f1 score (weighted): 0.8279392143888279
    run id: 1
    start: 1661540186.0719717
    time taken: 6.020878076553345
-   end: 1661540198.1268275
    result:
    -   accuracy: 0.8086785009861933
        f1 score (macro): 0.8123182018199305
        f1 score (micro): 0.8086785009861933
        f1 score (weighted): 0.8050388001524559
    run id: 2
    start: 1661540192.0928512
    time taken: 6.033976316452026
-   end: 1661540204.1166441
    result:
    -   accuracy: 0.8234714003944773
        f1 score (macro): 0.8265537857077874
        f1 score (micro): 0.8234714003944773
        f1 score (weighted): 0.8203890150811671
    run id: 3
    start: 1661540198.1268291
    time taken: 5.98981499671936



In [19]:
result = multi_run_experiment(train_dataset_fft_stats, test_dataset_fft_stats)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661540205.5365205
    result:
    -   accuracy: 0.8589743589743589
        f1 score (macro): 0.8572100184577393
        f1 score (micro): 0.8589743589743589
        f1 score (weighted): 0.8607386994909787
    run id: 1
    start: 1661540204.1478732
    time taken: 1.3886473178863525
-   end: 1661540206.9141402
    result:
    -   accuracy: 0.8579881656804734
        f1 score (macro): 0.8560504840908014
        f1 score (micro): 0.8579881656804734
        f1 score (weighted): 0.8599258472701451
    run id: 2
    start: 1661540205.5365222
    time taken: 1.3776180744171143
-   end: 1661540208.2941043
    result:
    -   accuracy: 0.8579881656804734
        f1 score (macro): 0.8565747242101258
        f1 score (micro): 0.8579881656804734
        f1 score (weighted): 0.8594016071508208
    run id: 3
    start: 1661540206.9141414
    time taken: 1.3799629211425781



In [20]:
result = multi_run_experiment(combined_train_dset, combined_test_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661540214.1558535
    result:
    -   accuracy: 0.8609467455621301
        f1 score (macro): 0.8599933854666691
        f1 score (micro): 0.8609467455621301
        f1 score (weighted): 0.8619001056575913
    run id: 1
    start: 1661540208.3249552
    time taken: 5.830898284912109
-   end: 1661540220.0297089
    result:
    -   accuracy: 0.868836291913215
        f1 score (macro): 0.8674618347828805
        f1 score (micro): 0.868836291913215
        f1 score (weighted): 0.8702107490435497
    run id: 2
    start: 1661540214.1558554
    time taken: 5.873853445053101
-   end: 1661540225.8737693
    result:
    -   accuracy: 0.8816568047337278
        f1 score (macro): 0.8806607772475133
        f1 score (micro): 0.8816568047337278
        f1 score (weighted): 0.8826528322199423
    run id: 3
    start: 1661540220.0297103
    time taken: 5.844058990478516



In [21]:
result = multi_run_experiment(resampled_train_dataset_fft_stats, resampled_test_dataset_fft_stats)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661540230.5869005
    result:
    -   accuracy: 0.8254437869822485
        f1 score (macro): 0.8284157533102537
        f1 score (micro): 0.8254437869822485
        f1 score (weighted): 0.8224718206542434
    run id: 1
    start: 1661540225.9048452
    time taken: 4.682055234909058
-   end: 1661540235.3579235
    result:
    -   accuracy: 0.8303747534516766
        f1 score (macro): 0.8330160299761332
        f1 score (micro): 0.8303747534516766
        f1 score (weighted): 0.82773347692722
    run id: 2
    start: 1661540230.5869024
    time taken: 4.771021127700806
-   end: 1661540240.09158
    result:
    -   accuracy: 0.8362919132149902
        f1 score (macro): 0.8396827610663943
        f1 score (micro): 0.8362919132149902
        f1 score (weighted): 0.8329010653635859
    run id: 3
    start: 1661540235.3579252
    time taken: 4.733654737472534

